In [ ]:
%%html
<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at 'In [3]'.</span>

An Exception was encountered at 'In [3]'.

In [1]:
import requests
import json
import pandas as pd
import datetime
import sqlite3

## Get Site Info from API

In [2]:
key1 = 'cUEwsckBq7bZ0wWCZL4jYMIGQicGUKd7'
site = 'https://api.meh.com/1/current.json?apikey=' + key1

In [3]:
current_dt = datetime.datetime.now()
current_date = str(current_dt.date())
current_time = str(current_dt.time())
r = requests.get(site)
status_code = r.status_code
print(current_dt)

ConnectionError: HTTPSConnectionPool(host='api.meh.com', port=443): Max retries exceeded with url: /1/current.json?apikey=cUEwsckBq7bZ0wWCZL4jYMIGQicGUKd7 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x6e1f5130>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))

In [ ]:
assert(status_code == 200), "Status code != 200, Status code: "+str(status_code)

In [ ]:
site_data = r.json()

## Save Raw Response 

In [ ]:
db_location = '../data/meh_scraper_qa2.db'

In [ ]:
# Parameters
db_location = "data/meh_scraper.db"


In [ ]:
con = sqlite3.connect(db_location)
cursor = con.cursor()

In [ ]:
raw_data_series = pd.Series([json.dumps(site_data)], index=[current_date], name='raw_response')
raw_data_series.index.name = 'date'

In [ ]:
raw_data_series.to_sql('raw_response_backup', con, if_exists='append', index=True)

In [ ]:
con.commit()

## Begin Parsing JSON

In [ ]:
def flatten_dict(parsed_site_series, k1):
    final_dict = {}
    for k,v in parsed_site_series.items():
        if k1 !='':
            new_k = k1 + '_' + k
        else:
            new_k = k

        if type(v) in [str, int, float]:
            final_dict[new_k] = v
        elif type(v) == dict:
            temp = flatten_dict(v, new_k)
            final_dict.update(temp)
        else:
            print(k + ' is ' + str(type(v)))
            pass
    return(final_dict)            

In [ ]:
site_data_dict = flatten_dict(site_data, '')            
site_data_dict['date'] = current_date
site_data_dict['time'] = current_time
site_data_dict.keys()

In [ ]:
# print(json.dumps(site_data_dict, indent=4))

In [ ]:
parsed_site_series = pd.DataFrame(site_data_dict, index=[1])
new_cols = parsed_site_series.columns

## Check matching table formats

In [ ]:
existing_table = pd.read_sql_query("select * from products limit 1"
                                  , con)
existing_cols = existing_table.columns

In [ ]:
new_table_missing_cols = set(existing_cols).difference(set(new_cols))
existing_table_missing_cols = set(new_cols).difference(set(existing_cols))

In [ ]:
def are_matching_columns(df1, df2, verbose=True):
    """
    Ensure two dataframes have the same columns 
    Returns True if matching, False if not
    """
    cols1 = set(df1.columns)
    cols2 = set(df2.columns)
    
    cols1_not2 = cols1.difference(cols2)
    cols2_not1 = cols2.difference(cols1)
    sym_diff = cols1.symmetric_difference(cols2)
    
    if verbose: 
        print("Columns not in df2: ", cols1_not2)
        print("Columns not in df1: ", cols2_not1)    
    
    output = True if sym_diff == set() else False 
    
    return(output)
    

In [ ]:
if len(new_table_missing_cols) != 0:
    for col in new_table_missing_cols:
        parsed_site_series[col] = None
    if are_matching_columns(parsed_site_series, existing_table):
        parsed_site_series.to_sql('products', con, if_exists='append', index=False)


In [ ]:
if len(existing_table_missing_cols) != 0:
    existing_table = pd.read_sql_query("select * from products"
                                  , con)
    for col in existing_table_missing_cols:
        existing_table[col] = None
    if are_matching_columns(parsed_site_series, existing_table):
        existing_table.to_sql('products', con, if_exists='replace', index=False)

In [ ]:
con.commit()
con.close()

## TODOs  

* logic for items, photos, launches, answers  
* Special page scanner to record info on bottom of page multiple times during the day, maybe include topics, votes and other info more frequently as well
* Links for external and tags accoringly 